In [1]:
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.simplefilter('ignore')

In [2]:
load_dotenv(find_dotenv('file.env'))

True

In [3]:
import openai,langchain
print("langchain->",langchain.__version__, "openai->",openai.__version__)

langchain-> 0.2.8 openai-> 1.35.14


# Role  Prompting

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. To work with role prompts, you could iteratively:

1- Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."

2 - Use the prompt to generate an output from an LLM.

3 - Analyze the generated response and, if necessary, refine the prompt for better results.

In [5]:
# In this example, the LLM is asked to act as a futuristic robot band 
# conductor and suggest a song title related to the given theme and year

from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature=0)

template = """"
            As a futuristic robot band conductor, I need you to help me come up with a song title.
            What's a cool song title for a song about {theme} in the year {year}?
"""

prompt = PromptTemplate(
    input_variables = ["theme", "year"],
    template = template)

input_data = {"theme":"interstellar travel","year":"3030"}

chain = LLMChain(llm = llm, prompt=prompt)

response = chain.invoke(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response['text'])

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Galactic Odyssey: Journey to 3030"


This is a good prompt for several reasons:

- **Clear instructions:** The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.

- **Specificity:** The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.

- **Open-ended creativity:** The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.
Focus on the task: The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

These elements help the LLM understand the user's intention and generate a suitable response.

# Few Shot Prompting

In [7]:
# Few Shot Prompting In the next example, the LLM is asked to provide the emotion 
# associated with a given color based on a few examples of color-emotion pairs:
from langchain import FewShotPromptTemplate
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=  ["input"],
    example_separator = "\n")

formatted_prompt = few_shot_prompt.format(input="purple")

# The formatted prompt is not passed directly to the chain but type casted as a PromptTemplate object
chain = LLMChain(llm = llm, prompt=PromptTemplate(template=formatted_prompt))

response  = chain.invoke({})

print("Colour","purple")
print("Emotion",response['text'])

Colour purple
Emotion  royalty


# Bad Prompt practices

In [8]:
#  a too-vague prompt that provides little context or guidance for the model to generate a meaningful response.

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="dogs")

'Tell me something about dogs.'

# Chain Prompting

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

**PromptTemplate** class makes constructing prompts with dynamic inputs easier. This is useful when creating a prompt chain that depends on previous answers. 

In [11]:
# prompt 1
template_question = """What is the anme of the famous scientist who developed the theory of general relativity?
Answer:"""

question_prompt = PromptTemplate(input_variabls=[], template = template_question)

# prompt 2

template_fact = """Provide a brief description of {scientist}'s thoery of general relativity.
Answer:
"""

fact_prompt = PromptTemplate(input_variables=['scientist'], template=template_fact)

# create llmchain for the first prompt

chain_question = LLMChain(llm=llm, prompt=question_prompt)

# running with empty input

response_question = chain_question.run({})

# extracting name from response
scientist = response_question.strip()

# create llmchain for the 2nd prompt

chain_fact = LLMChain(llm=llm, prompt = fact_prompt)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print()
print("Fact:", response_fact)

Scientist: Albert Einstein

Fact: Albert Einstein's theory of general relativity is a theory of gravity that explains how massive objects interact with each other and how they affect the fabric of space and time. According to this theory, gravity is not a force between masses, but rather a curvature of space and time caused by the presence of massive objects. This curvature is what causes objects to move towards each other, giving the appearance of a gravitational force. General relativity also predicts the phenomenon of gravitational time dilation, where time moves slower in the presence of strong gravitational fields. This theory has been extensively tested and has been shown to accurately predict the behavior of objects in the universe, including the bending of light by massive objects and the existence of black holes. It is considered one of the most important and influential theories in modern physics.
